In [4]:
# Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import xgboost as xgb
import optuna


C:\Users\yhado\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load datasets with raw string paths
train = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\train.csv')
test = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\test.csv')
users = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\users.csv')
user_friends = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\user_friends.csv.gz', compression='gzip')
events = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\events.csv.gz', compression='gzip')
event_attendees = pd.read_csv(r'C:\Users\yhado\associationsltd.co.uk\OneDrive - associationsltd.co.uk\ML-AI\Final project\event-recommendation-engine-challenge\event_attendees.csv.gz', compression='gzip')


In [6]:
# Verify that data is loaded correctly
print("Train dataset:")
print(train.head())

print("Test dataset:")
print(test.head())

print("Users dataset:")
print(users.head())

print("User Friends dataset:")
print(user_friends.head())

print("Events dataset:")
print(events.head())

print("Event Attendees dataset:")
print(event_attendees.head())


Train dataset:
      user       event  invited                         timestamp  interested  \
0  3044012  1918771225        0  2012-10-02 15:53:05.754000+00:00           0   
1  3044012  1502284248        0  2012-10-02 15:53:05.754000+00:00           0   
2  3044012  2529072432        0  2012-10-02 15:53:05.754000+00:00           1   
3  3044012  3072478280        0  2012-10-02 15:53:05.754000+00:00           0   
4  3044012  1390707377        0  2012-10-02 15:53:05.754000+00:00           0   

   not_interested  
0               0  
1               0  
2               0  
3               0  
4               0  
Test dataset:
      user       event  invited                         timestamp
0  1776192  2877501688        0  2012-11-30 11:39:01.230000+00:00
1  1776192  3025444328        0  2012-11-30 11:39:01.230000+00:00
2  1776192  4078218285        0  2012-11-30 11:39:01.230000+00:00
3  1776192  1024025121        0  2012-11-30 11:39:01.230000+00:00
4  1776192  2972428928        0  2

In [7]:
# Test loading one file
train_file = 'C:\\Users\\yhado\\associationsltd.co.uk\\OneDrive - associationsltd.co.uk\\ML-AI\\Final project\\event-recommendation-engine-challenge\\train.csv'

try:
    train = pd.read_csv(train_file)
    print("File loaded successfully!")
    print(train.head())
except FileNotFoundError:
    print("File not found at:", train_file)
except pd.errors.ParserError:
    print("Error parsing the file:", train_file)


File loaded successfully!
      user       event  invited                         timestamp  interested  \
0  3044012  1918771225        0  2012-10-02 15:53:05.754000+00:00           0   
1  3044012  1502284248        0  2012-10-02 15:53:05.754000+00:00           0   
2  3044012  2529072432        0  2012-10-02 15:53:05.754000+00:00           1   
3  3044012  3072478280        0  2012-10-02 15:53:05.754000+00:00           0   
4  3044012  1390707377        0  2012-10-02 15:53:05.754000+00:00           0   

   not_interested  
0               0  
1               0  
2               0  
3               0  
4               0  


In [8]:
# Cell 3: Verify that data is loaded correctly
print(train.head())
print(test.head())
print(users.head())
print(events.head())


      user       event  invited                         timestamp  interested  \
0  3044012  1918771225        0  2012-10-02 15:53:05.754000+00:00           0   
1  3044012  1502284248        0  2012-10-02 15:53:05.754000+00:00           0   
2  3044012  2529072432        0  2012-10-02 15:53:05.754000+00:00           1   
3  3044012  3072478280        0  2012-10-02 15:53:05.754000+00:00           0   
4  3044012  1390707377        0  2012-10-02 15:53:05.754000+00:00           0   

   not_interested  
0               0  
1               0  
2               0  
3               0  
4               0  
      user       event  invited                         timestamp
0  1776192  2877501688        0  2012-11-30 11:39:01.230000+00:00
1  1776192  3025444328        0  2012-11-30 11:39:01.230000+00:00
2  1776192  4078218285        0  2012-11-30 11:39:01.230000+00:00
3  1776192  1024025121        0  2012-11-30 11:39:01.230000+00:00
4  1776192  2972428928        0  2012-11-30 11:39:21.985000+00:

In [4]:
# Cell 3: Preprocess the data
# Extract useful features from the events and users dataset
events['start_time'] = pd.to_datetime(events['start_time'])
train['timestamp'] = pd.to_datetime(train['timestamp'])

# Merge the train and user demographic data
train_merged = pd.merge(train, users, left_on='user', right_on='user_id', how='left')

# Handling missing values and creating relevant features
train_merged.fillna(0, inplace=True)

# Dropping unnecessary columns
train_merged.drop(columns=['timestamp', 'joinedAt', 'locale'], inplace=True)


NameError: name 'events' is not defined

In [5]:
# Cell 4: Feature engineering
# Create features for the event interactions (attendees, friends, etc.)
event_attendees['yes_count'] = event_attendees['yes'].str.split().apply(len)
event_attendees['no_count'] = event_attendees['no'].str.split().apply(len)
event_attendees['maybe_count'] = event_attendees['maybe'].str.split().apply(len)

# Merge event data with the train data
train_merged = pd.merge(train_merged, event_attendees[['event_id', 'yes_count', 'no_count', 'maybe_count']], left_on='event', right_on='event_id', how='left')
train_merged.fillna(0, inplace=True)


NameError: name 'event_attendees' is not defined

In [6]:
# Cell 5: Prepare training and test sets
X = train_merged.drop(columns=['interested', 'not_interested', 'user', 'event', 'event_id'])
y = train_merged['interested']

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


NameError: name 'train_merged' is not defined

In [7]:
# Cell 6: Define XGBoost model and Bayesian optimization function
def objective(trial):
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': 'gbtree',
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5)
    }

    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)

    # Predict on validation data
    y_pred = model.predict_proba(X_val)[:, 1]
    return average_precision_score(y_val, y_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))


NameError: name 'optuna' is not defined

In [16]:
# Cell 7: Train the final model with the best hyperparameters
best_params = study.best_trial.params
final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X, y)


In [17]:
# Cell 8: Make predictions on test set and output recommendations
test_merged = pd.merge(test, users, left_on='user', right_on='user_id', how='left')
test_merged = pd.merge(test_merged, event_attendees[['event_id', 'yes_count', 'no_count', 'maybe_count']], left_on='event', right_on='event_id', how='left')
test_merged.fillna(0, inplace=True)
X_test = test_merged.drop(columns=['user', 'event', 'event_id'])

# Predict probabilities
test_merged['interest_prob'] = final_model.predict_proba(X_test)[:, 1]

# Rank events per user by predicted interest
recommendations = test_merged.groupby('user').apply(lambda x: x.sort_values('interest_prob', ascending=False)['event'].tolist())

# Display top recommendations for each user
recommendations.head()


In [10]:
# Cell 9: Compute Mean Average Precision at 200 for validation set
def mean_average_precision(y_true, y_pred):
    return average_precision_score(y_true, y_pred)

y_pred_val = final_model.predict_proba(X_val)[:, 1]
map_score = mean_average_precision(y_val, y_pred_val)
print(f"Mean Average Precision at 200: {map_score}")


NameError: name 'final_model' is not defined

In [11]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: C:\Users\yhado
